## Data Description

This data represents the results of a large product testing study. For each `product_code` you are given a number of product `attributes` (fixed for the code) as well as a number of `measurement` values for each individual product, representing various lab testing methods. Each product is used in a simulated real-world environment experiment, and and absorbs a certain amount of fluid (`loading`) to see whether or not it fails. &nbsp;

Your task is to use the data to predict individual product failures of new codes with their individual lab test results.

## Evaluation

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

## Understanding

- Same product_code means they are the same products
- Same product consists of same attributes

## Blueprint

1. Numerize 'attribute_0' and 'attribute_1'
2. Drop the product code(A, B, C, D, E)
3. Apply PCA
4. Split into training and validating data
5. Apply ML models

### Import Packages

In [182]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn import metrics     # error function : metrics.roc_auc_score()

# Show all the columns and rows
pd.set_option("display.max_columns", None)  # columns
# pd.set_option("display.max_rows", None)   # rows

## 1. Data Loading

In [183]:
# Load dataset
data = pd.read_csv('train.csv') # training
te = pd.read_csv('test.csv')    # testing

print(data.shape)
data.head()

(26570, 26)


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [184]:
# Store 'id'
id = data.id

# Drop 'id'
data = data.drop(columns=['id'])

print(data.shape)
data.head()

(26570, 25)


,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


## 2. Data Exploration

In [185]:
# Check data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_code    26570 non-null  object 
 1   loading         26320 non-null  float64
 2   attribute_0     26570 non-null  object 
 3   attribute_1     26570 non-null  object 
 4   attribute_2     26570 non-null  int64  
 5   attribute_3     26570 non-null  int64  
 6   measurement_0   26570 non-null  int64  
 7   measurement_1   26570 non-null  int64  
 8   measurement_2   26570 non-null  int64  
 9   measurement_3   26189 non-null  float64
 10  measurement_4   26032 non-null  float64
 11  measurement_5   25894 non-null  float64
 12  measurement_6   25774 non-null  float64
 13  measurement_7   25633 non-null  float64
 14  measurement_8   25522 non-null  float64
 15  measurement_9   25343 non-null  float64
 16  measurement_10  25270 non-null  float64
 17  measurement_11  25102 non-null 

In [186]:
# Check the distribution
data["failure"].value_counts()  # target
data["product_code"].value_counts() # product code

C    5765
E    5343
B    5250
D    5112
A    5100
Name: product_code, dtype: int64

In [187]:
# # Opt A. Drop missing values
# data = data.dropna()

# Opt B. Replace missing values with 0
data = data.fillna(0)

# Opt C. Replace missing values with the feature's mean

C    5765
E    5343
B    5250
D    5112
A    5100
Name: product_code, dtype: int64

In [ ]:
# Check the distribution again
data["failure"].value_counts()     # target
data["product_code"].value_counts()    # product code

### - Objective Values

In [188]:
# # Unique values in each column
# data['product_code'].unique()   # array(['A', 'B', 'C', 'D', 'E'], dtype=object)
# data['attribute_0'].unique()    # array(['material_7', 'material_5'], dtype=object)
# data['attribute_1'].unique()    # array(['material_8', 'material_5', 'material_6'], dtype=object)

In [189]:
# Check attribute combinations for each product
def combinations(df):
    products = df['product_code'].unique()      # product codes
    attr = []       # list of the combination of attributes

    for product in products:
        attr = []
        subset = df.loc[df['product_code']==product, :]     # get subsets for each 'product code'

        attr.append(subset['attribute_0'].unique())
        attr.append(subset['attribute_1'].unique())
        attr.append(subset['attribute_2'].unique())
        attr.append(subset['attribute_3'].unique())

        print("Product",product, "consists of", attr)

In [190]:
# combinations(data)

In [191]:
# combinations(te)

### - Int/Float Values

In [192]:
data.describe()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
count,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000
mean,126.623502,6.754046,7.240459,7.415883,8.232518,6.256568,17.536406,11.494434,16.692034,16.986161,11.303433,18.274322,10.902855,15.329114,18.112822,10.997322,14.607806,14.916537,13.861717,15.153533,640.986841,0.212608
std,40.759151,1.471852,1.456493,4.116690,4.199401,3.309109,2.337115,1.924252,2.870842,3.142212,2.374236,3.832824,2.589918,3.737513,4.622983,3.136565,4.063576,4.353575,4.235109,4.743193,229.212732,0.409160
min,0.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,99.285000,6.000000,6.000000,4.000000,5.000000,4.000000,17.083250,11.002000,16.379000,16.768000,10.955000,18.237000,10.637250,15.041000,17.941000,10.473000,14.697250,14.790000,13.650000,14.907000,588.508750,0.000000
50%,121.970000,6.000000,8.000000,7.000000,8.000000,6.000000,17.768000,11.707000,17.097000,17.479000,11.668000,18.966000,11.369000,16.042500,19.106000,11.595000,15.530000,15.895000,14.806000,16.239000,686.710000,0.000000
75%,148.837500,8.000000,8.000000,10.000000,11.000000,8.000000,18.456000,12.396000,17.785000,18.155750,12.363750,19.676000,12.065000,16.975000,20.144000,12.643000,16.306000,16.997000,15.917750,17.514000,774.633750,0.000000
max,385.860000,9.000000,9.000000,29.000000,29.000000,24.000000,21.499000,16.484000,21.425000,21.543000,15.419000,23.807000,15.412000,22.479000,25.640000,17.663000,22.713000,22.303000,21.626000,24.094000,1312.794000,1.000000


## 3. Preprocessing

### (1) Numerize 'attribute_0' and 'attribute_1'

In [193]:
# Apply function element-wise
numerized_attr01 = data.loc[:, ['attribute_0','attribute_1']].applymap(lambda x: int(x[9]))

In [194]:
# Drop 'attribute_0', 'attribute_1' and concat
data = data.drop(columns=['attribute_0', 'attribute_1'])
data = pd.concat([numerized_attr01, data], axis=1)

In [195]:
print(data.shape)
data.head()

(26570, 25)


,attribute_0,attribute_1,product_code,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,7,8,A,80.10,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,0.000,13.034,14.684,764.100,0
1,7,8,A,84.89,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,7,8,A,82.43,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,0.000,13.798,16.711,18.631,14.094,17.946,663.376,0
3,7,8,A,101.07,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,7,8,A,188.06,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [196]:
# Double check the combination
combinations(data)

Product A consists of [array([7]), array([8]), array([9]), array([5])]
Product B consists of [array([5]), array([5]), array([8]), array([8])]
Product C consists of [array([7]), array([8]), array([5]), array([8])]
Product D consists of [array([7]), array([5]), array([6]), array([6])]
Product E consists of [array([7]), array([6]), array([6]), array([9])]


### (2) Drop 'product_code'(A, B, C, D, E)

In [197]:
# Drop 'product_code'
data = data.drop(columns=['product_code'])

print(data.shape)
data.head()

(26570, 24)


,attribute_0,attribute_1,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,7,8,80.10,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,0.000,13.034,14.684,764.100,0
1,7,8,84.89,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,7,8,82.43,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,0.000,13.798,16.711,18.631,14.094,17.946,663.376,0
3,7,8,101.07,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,7,8,188.06,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


### (3). Apply PCA

In [198]:
X = data.iloc[:, 0:23]
y = data.loc[:, 'failure']

In [199]:
# Mean Centering
centered = X - X.mean()    # Uncomment for Numerzied data B
centered.head()

,attribute_0,attribute_1,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,0.395183,1.572149,-46.523502,2.245954,-2.240459,-0.415883,-0.232518,-2.256568,0.503594,1.023566,-0.944034,2.305839,0.435567,1.880678,-0.230855,0.529886,-0.518822,4.195678,0.421194,-14.916537,-0.827717,-0.469533,123.113159
1,0.395183,1.572149,-41.733502,2.245954,-2.240459,6.584117,-5.232518,-3.256568,0.676594,0.045566,1.024966,0.906839,1.444567,-0.385322,1.545145,2.617886,-0.197822,0.757678,0.124194,0.508463,0.533283,0.477467,41.070159
2,0.395183,1.572149,-44.193502,2.245954,-2.240459,4.584117,-7.232518,-1.256568,0.520594,0.157566,0.045966,1.253839,1.414567,0.013678,1.812145,0.277886,-18.112822,2.800678,2.103194,3.714463,0.232283,2.792467,22.389159
3,0.395183,1.572149,-25.553502,2.245954,-2.240459,5.584117,-6.232518,-0.256568,-0.241406,-0.306434,1.883966,1.352839,1.279567,0.785678,1.568145,1.016886,0.264178,-0.977322,0.642194,0.645463,2.292283,2.018467,185.295159
4,0.395183,1.572149,61.436498,2.245954,-2.240459,1.584117,-6.232518,1.743432,1.809594,1.455566,0.297966,-1.240161,0.002567,-0.181322,-0.565855,1.752886,1.819178,1.430678,1.574194,-2.156537,-0.708717,1.258467,-61.101841


In [200]:
# Check Variance
print(centered.var())

attribute_0           0.634220
attribute_1           1.842681
loading            1661.308375
attribute_2           2.166348
attribute_3           2.121371
measurement_0        16.947138
measurement_1        17.634969
measurement_2        10.950204
measurement_3         5.462105
measurement_4         3.702745
measurement_5         8.241733
measurement_6         9.873494
measurement_7         5.636996
measurement_8        14.690542
measurement_9         6.707676
measurement_10       13.969001
measurement_11       21.371968
measurement_12        9.838043
measurement_13       16.512648
measurement_14       18.953616
measurement_15       17.936147
measurement_16       22.497877
measurement_17    52538.476551
dtype: float64


In [201]:
# Covariance matrix
cov = np.dot(centered.T, centered)/(len(centered)-1)
df_cov = pd.DataFrame(cov)

In [202]:
# Eigenvalues & Eigenvectors
eig = np.linalg.eig(cov)
# print('<eigenvalues>\n', eig[0],'\n')
# print('<eigenvectors>\n', eig[1])

In [203]:
# Contribution in the Data
def percent_variation(n):
    '''Percent Variation of top 'n' factors'''
    trace = sum(eig[0])    # sum of all eigenvalues
    contribution = [np.round(i/trace,5) for i in eig[0]]
    return contribution[:n]    # return the first n number of contributions

In [204]:
n = 2
print(percent_variation(n))
print('Total contribution of 2 components:', sum(percent_variation(n))*100, '%\n')

[0.9653, 0.03052]
Total contribution of 2 components: 99.58200000000001 %



In [205]:
# Fit to PCA (getting a projection matrix)
pca = PCA(n_components=2)
pca.fit(X)

PCA(n_components=2)

In [206]:
# How much each of the features influences the PC
influence = pd.DataFrame(pca.components_, columns=list(X.columns))
influence

,attribute_0,attribute_1,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,-6.858384e-08,0.000061,0.000829,0.000022,-0.000003,0.000099,-0.000095,-0.000023,0.000210,0.000334,0.001012,0.000692,0.000664,0.001083,0.000181,0.000192,0.000147,-0.000097,-0.000083,0.000004,-0.000049,0.000008,0.999998
1,7.101038e-05,-0.000013,0.999998,-0.000258,-0.000044,-0.000482,0.000304,0.000071,0.000104,-0.000260,-0.000156,0.000708,0.000229,-0.000572,-0.000129,-0.000490,0.000964,-0.000327,0.000405,0.000534,0.000251,-0.000010,-0.000829


In [207]:
# Projection (transforming the original data via projection matrix)
data_pca = pca.transform(X)
data_pca = pd.DataFrame(data_pca, columns = ['PC1', 'PC2']) # rename

print(data_pca.shape)
data_pca.head()

(26570, 2)


,PC1,PC2
0,123.077443,-46.635487
1,41.039862,-41.773320
2,22.353120,-44.231631
3,185.279754,-25.710537
4,-61.049528,61.482801


## 4. Apply ML models

In [208]:
# Split into training and validating
X_train, X_val, y_train, y_val = train_test_split(data_pca, y, test_size=0.33, 
                                                    random_state=0, stratify=y)

### (1) Linear SVC

In [209]:
from sklearn.svm import LinearSVC

LinSVC = LinearSVC(random_state=0)
LinSVC.fit(X_train, y_train)
y_LinSVC = LinSVC.predict(X_val)

/Users/lucy/Library/Python/3.8/lib/python/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [210]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_LinSVC)

0.5390986813685424

### (2) SGD Classifier

In [211]:
from sklearn.linear_model import SGDClassifier

SGD = SGDClassifier(random_state=0)
SGD.fit(X_train, y_train)
y_SGD = SGD.predict(X_val)

In [212]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_SGD)

0.530298222660074

### (3) KNN Classifier

In [213]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(X_train, y_train)
y_KNN = KNN.predict(X_val)

In [214]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_KNN)

0.5025630646449516

### (4) Kernel Approximation

In [219]:
# check

### (5) SVC

In [215]:
from sklearn.svm import SVC

SVC = SVC(random_state=0)
SVC.fit(X_train, y_train)
y_SVC = SVC.predict(X_val)

In [216]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_SVC)

0.5

### (6) Ensemble Classifiers

In [217]:
from sklearn.ensemble import RandomForestRegressor

Ensb = RandomForestRegressor(max_depth=5, random_state=0)
Ensb.fit(X_train, y_train)
y_Ensb = SVC.predict(X_val)

In [218]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_Ensb)

0.5

## 5. Apply the best-fit model to the testing data